In [0]:
!wget https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip

--2019-01-23 04:38:57--  https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.190
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 311442766 (297M) [application/zip]
Saving to: ‘flower_data.zip’

flower_data.zip     100%[===================>] 297.01M  36.3MB/s    in 8.8s    

2019-01-23 04:39:06 (33.7 MB/s) - ‘flower_data.zip’ saved [311442766/311442766]



In [0]:
!unzip flower_data.zip

In [0]:
!ls

data  flower_data  flower_data.zip  model_cifar.pt  sample_data


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision.models as models

In [0]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_data = datasets.ImageFolder(valid_dir, transform=validation_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
vloader = torch.utils.data.DataLoader(validation_data, batch_size =32,shuffle = True)

In [0]:
def nn_setup(dropout=0.5, hidden_layer1 = 120,lr = 0.001):
     
    model = models.vgg16(pretrained=True)        
        
    for param in model.parameters():
        param.requires_grad = False

        from collections import OrderedDict
        classifier = nn.Sequential(OrderedDict([
            ('dropout',nn.Dropout(dropout)),
            ('inputs', nn.Linear(25088, hidden_layer1)),
            ('relu1', nn.ReLU()),
            ('hidden_layer1', nn.Linear(hidden_layer1, 90)),
            ('relu2',nn.ReLU()),
            ('hidden_layer2',nn.Linear(90,80)),
            ('relu3',nn.ReLU()),
            ('hidden_layer3',nn.Linear(80,102)),
            ('output', nn.LogSoftmax(dim=1))
                          ]))
        
        
        model.classifier = classifier
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.classifier.parameters(), lr )
        model.cuda()
        
        return model , optimizer ,criterion 

In [0]:
model,optimizer,criterion = nn_setup()

In [0]:
epochs = 12
print_every = 5
steps = 0
loss_show=[]

# change to cuda
model.to('cuda')

for e in range(epochs):
    running_loss = 0
    for ii, (inputs, labels) in enumerate(trainloader):
        steps += 1
        
        inputs,labels = inputs.to('cuda'), labels.to('cuda')
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            vlost = 0
            accuracy=0
            
            
            for ii, (inputs2,labels2) in enumerate(vloader):
                optimizer.zero_grad()
                
                inputs2, labels2 = inputs2.to('cuda:0') , labels2.to('cuda:0')
                model.to('cuda:0')
                with torch.no_grad():    
                    outputs = model.forward(inputs2)
                    vlost = criterion(outputs,labels2)
                    ps = torch.exp(outputs).data
                    equality = (labels2.data == ps.max(1)[1])
                    accuracy += equality.type_as(torch.FloatTensor()).mean()
                    
            vlost = vlost / len(vloader)
            accuracy = accuracy /len(vloader)
            
                    
            
            print("Epoch: {}/{}... ".format(e+1, epochs),
                  "Loss: {:.4f}".format(running_loss/print_every),
                  "Validation Lost {:.4f}".format(vlost),
                   "Accuracy: {:.4f}".format(accuracy))
            
            
            running_loss = 0


Epoch: 1/12...  Loss: 4.6014 Validation Lost 0.1783 Accuracy: 0.0274
Epoch: 1/12...  Loss: 4.5824 Validation Lost 0.1705 Accuracy: 0.0610
Epoch: 1/12...  Loss: 4.3922 Validation Lost 0.1700 Accuracy: 0.0947
Epoch: 1/12...  Loss: 4.3230 Validation Lost 0.1609 Accuracy: 0.0966
Epoch: 1/12...  Loss: 4.1081 Validation Lost 0.1482 Accuracy: 0.1660
Epoch: 1/12...  Loss: 4.1800 Validation Lost 0.1541 Accuracy: 0.1680
Epoch: 1/12...  Loss: 3.9845 Validation Lost 0.1400 Accuracy: 0.2321
Epoch: 1/12...  Loss: 3.7720 Validation Lost 0.1040 Accuracy: 0.2388
Epoch: 1/12...  Loss: 3.5137 Validation Lost 0.1178 Accuracy: 0.2808
Epoch: 1/12...  Loss: 3.3270 Validation Lost 0.1216 Accuracy: 0.3391
Epoch: 1/12...  Loss: 3.2687 Validation Lost 0.1154 Accuracy: 0.3503
Epoch: 1/12...  Loss: 3.0551 Validation Lost 0.0911 Accuracy: 0.3582
Epoch: 1/12...  Loss: 2.9140 Validation Lost 0.0815 Accuracy: 0.4236
Epoch: 1/12...  Loss: 2.7306 Validation Lost 0.0739 Accuracy: 0.4442
Epoch: 1/12...  Loss: 2.6512 Valid

In [0]:
model.class_to_idx = train_data.class_to_idx
model.cpu
torch.save({'structure' :'vgg16',
            'hidden_layer1':120,
            'state_dict':model.state_dict(),
            'class_to_idx':model.class_to_idx},
            'checkpoint.pth')

In [0]:
!ls

checkpoint.pth	data  flower_data  flower_data.zip  model_cifar.pt  sample_data


In [0]:
model.class_to_idx = train_data.class_to_idx
model.cpu
torch.save({'structure' :'vgg16',
            'hidden_layer1':120,
            'state_dict':model.state_dict(),
            'class_to_idx':model.class_to_idx},
            'checkpoint.pth')

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'checkpoint.pth'})
uploaded.SetContentFile('checkpoint.pth')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1oUVfbpehWdJluJ_DJL_l-JvdzJDJgKcB


NameError: ignored